In [1]:
import numpy as np 
import pandas as pd
import datetime
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
import warnings

'''missing data interpolation'''
from scipy.interpolate import CubicSpline

warnings.filterwarnings('ignore')
sns.set_style('darkgrid')
pd.set_option('display.max_columns', None)
''' reading data '''
df = pd.read_csv('energy_dataset.csv')
''' displaying first five rows '''
df.head()
''' shape of data '''
df.shape
''' checking null values '''
for column in df.columns:
    if df[column].isnull().sum() > 0:
        print(column, df[column].isnull().sum())
'''info of data '''
df.info()
''' dropping columns '''
df.drop(['generation hydro pumped storage aggregated', 'forecast wind offshore eday ahead', 'total load forecast', 
         'forecast solar day ahead','forecast wind onshore day ahead', 'price day ahead'], axis=1, inplace=True)
df.head()
df.shape
''' rename columns '''
df.rename(columns=lambda x: x[11:] if 'generation' in x else x, inplace=True)
df.rename(columns={'total load actual':'energy demand'}, inplace=True)
df.head()
''' Create different time granularity '''

''' converting time into date time '''
df['time'] = pd.to_datetime(df['time'], utc=True)

''' creating new column '''
df['Year-Month'] = pd.to_datetime(df['time'].dt.strftime('%Y-%m-01')).dt.date
''' Missing value analysis '''
df[df.isna().any(axis=1)]
''' swarm plot '''
plt.figure(figsize=(10, 5))
sns.swarmplot(x='time', data=df[df.isna().any(axis=1)])
plt.xticks(rotation=45)
plt.xlabel('time', fontsize=15)
plt.title('NaN values across time', fontsize=20);
''' groupby '''
df[df.isna().any(axis=1)].groupby('Year-Month')['time'].count()
''' Consolidate different energy sources '''

fossil_fuel = ['fossil brown coal/lignite', 'fossil gas', 'fossil hard coal', 'fossil oil']
hydro = ['hydro pumped storage consumption', 'hydro run-of-river and poundage', 'hydro water reservoir']
wind = ['wind onshore']
others = ['other', 'other renewable'] 
df['fossil_fuel'] = df.loc[:, fossil_fuel].sum(axis=1)
df['hydro'] = df.loc[:, hydro].sum(axis=1)
df['wind'] = df.loc[:, wind].sum(axis=1)
df['others'] = df.loc[:, others].sum(axis=1)
energy_ls = ['biomass','fossil_fuel','hydro','wind','solar','nuclear','waste','others']
energy_f = pd.melt(df, id_vars='Year-Month', value_vars=energy_ls)
df_ = energy_f.loc[energy_f.value == 0]
df_.sort_values(by='Year-Month', inplace=True)

''' swarm plot '''
sns.swarmplot(x='Year-Month', y='variable', data=df_)
plt.title('NaN Across Time Group By Energy Source', fontsize=20)
plt.ylabel('Energy Sources', fontsize=15)
plt.xlabel('Time (Year-Month)', fontsize=15)
plt.xticks(rotation=45);
df[energy_ls] = df[energy_ls].replace(['0', 0], np.nan)
''' interpolation '''
def inter_pol(col_limit, energy):
    for column in col_limit:
        index_list = df[df[column].isna()].index
        for index in index_list:
            lb = index - 10
            up = index + 10
            if ((df.iloc[lb:up][column].isna()).sum() > 1):
                temp = df.iloc[lb:up][column]
                indices = temp[temp.isna()].index
                temp = temp.interpolate(method='quadratic', order=4)
                for index in indices:
                    energy.loc[index, column] = temp.loc[index]
            else:
                lb = index - 2
                up = index + 2
                temp = df.iloc[lb: up][column]
                temp = temp[~temp.isna()]
                X = temp.index.values
                y = temp.values
                cs = CubicSpline(X , y)
                df.loc[index, column] = cs(index).item(0)
inter_pol(energy_ls, df)
df.loc[:,energy_ls].isna().sum() + df.loc[:,energy_ls][df.loc[:,energy_ls] == 0].sum()

